## Definição do *dataset*

O *dataset utilizado será o "TOWARDS LIMB POSITION INVARIANT MYOELECTRIC PATTERN RECOGNITION USING TIME-DEPENDENT SPECTRAL FEATURES" [1]. Maiores informações podem ser vistas no site: https://www.rami-khushaba.com/electromyogram-emg-repository.html

De acordo com a figura seguinte, neste *dataset* existem 5 posições e 8 movimentos. Algumas questões de projetos foram levadas em consideração:
1. A primeira posição (P1) possui 40.000 *samples* e todas as outras possuem 20.000 *samples*. Desta forma, a posição 1 foi **excluída** do experimento de teste;
2. Cada posição possui 5 tentativas. Desta forma, será utilizada todas as tentativas do mesmo movimento como uma soma total de *samples* (todas as tentativas serão concatenadas);
3. Como este experimento resultará em 4 posições e 8 movimentos, será considerado primeiramente um experimento com 32 classes (4 * 8) e um segundo experimento com 8 classes (somando/dispresando as posições do braço).

[1] R. N. Khushaba, Maen Takruri, Jaime Valls Miro, and Sarath Kodagoda, "Towards limb position invariant myoelectric pattern recognition using time-dependent spectral features", Neural Networks, vol. 55, pp. 42-58, 2014. https://doi.org/10.1016/j.neunet.2014.03.010

### Separação das classes

In [71]:
from glob import glob

# exclusão dos arquivos com "Pos1" no nome
arquivos = glob("datasets/EMG_data/Pos[2-5]*.txt")
# índices para as 32 classes considerando as posições de 2 à 5
cl32 = {
    "Pos2_HandOpen": list(), "Pos3_HandOpen": list(), "Pos4_HandOpen": list(), "Pos5_HandOpen": list(),
    "Pos2_HandRest": list(), "Pos3_HandRest": list(), "Pos4_HandRest": list(), "Pos5_HandRest": list(),
    "Pos2_ObjectGrip": list(), "Pos3_ObjectGrip": list(), "Pos4_ObjectGrip": list(), "Pos5_ObjectGrip": list(),
    "Pos2_PichGrip": list(), "Pos3_PichGrip": list(), "Pos4_PichGrip": list(), "Pos5_PichGrip": list(),
    "Pos2_WristExten": list(), "Pos3_WristExten": list(), "Pos4_WristExten": list(), "Pos5_WristExten": list(),
    "Pos2_WristFlex": list(), "Pos3_WristFlex": list(), "Pos4_WristFlex": list(), "Pos5_WristFlex": list(),
    "Pos2_WristPron": list(), "Pos3_WristPron": list(), "Pos4_WristPron": list(), "Pos5_WristPron": list(),
    "Pos2_WristSupi": list(), "Pos3_WristSupi": list(), "Pos4_WristSupi": list(), "Pos5_WristSupi": list(),
}
# índices para as 8 classes desconsiderando as posições de 2 à 5
cl08 = {
    "HandOpen": list(),
    "HandRest": list(),
    "ObjectGrip": list(),
    "PichGrip": list(),
    "WristExten": list(),
    "WristFlex": list(),
    "WristPron": list(),
    "WristSupi": list(),
}

## Carregamento do *dataset*

Neste trecho de código, todos as amostras serão dividas entre os conjuntos de calsses específicas

In [72]:
import re

for arquivo in arquivos:
    trial_file = open(arquivo)
    nome = trial_file.name.split('/')[-1].split('.')[0]
    trial = list()
    for linha in trial_file.readlines():
        # foi necessário substituir os números "int" por "float"
        linha = re.sub(r"(?<=\ )(\d)(?=\ )", r"\1.0", linha)
        # casamento da linha com 7 pontos (1 para cada eletrodo)
        sample = [float(s) for s in re.findall(r"\-?\d\.\d+", linha)]
        trial.append(sample)
    cl08["{}".format(nome[5:-3])].append(trial)
    cl32["Pos{}{}".format(nome[3], nome[4:-3])].append(trial)

Agora será realizado a divisão dos dados e suas respectivas classes em sequência, formando dados em `numpy.array` para X1 (dados EMG das 8 classes), y1 (*labels* das 8 classes), X2 (dados EMG das 32 classes), y2 (*labels* das 32 classes)

In [73]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

X1 = np.array(list(cl08.values()))
X2 = np.array(list(cl32.values()))
# transformando labels categóricos em numéricos
le = LabelEncoder()
y1 = np.array(le.fit_transform(list(cl08.keys())))
y2 = np.array(le.fit_transform(list(cl32.keys())))
# shape: [8 classes, 24 trials, 20000 samples, 7 eletrodos]
print(X1.shape)
# shape: [32 classes, 6 trials, 20000 samples, 7 eletrodos]
print(X2.shape)
print(y1)
print(y2)

(8, 24, 20000, 7)
(32, 6, 20000, 7)
[0 1 2 3 4 5 6 7]
[ 0  8 16 24  1  9 17 25  2 10 18 26  3 11 19 27  4 12 20 28  5 13 21 29
  6 14 22 30  7 15 23 31]


É necessário inverter os eixos dos "samples" e dos "trials"

In [74]:
X1 = X1.swapaxes(2, 3)
X2 = X2.swapaxes(2, 3)
print(X1.shape)
print(X2.shape)

(8, 24, 7, 20000)
(32, 6, 7, 20000)


## Extração de características

Vamos agora aplicar a extração de características dos sinais. A titulo de exemplificação, daqui pra frente iremos trabalhar apenas com os dados com 8 classes (`X1` e `y1`).

Serão extraídas 4 características como exemplo. 2 no domímio do tempo (`VAR` e `RMS`) e 2 no domínio da frequência (`FMD` e `MMDF`):
* `Variance of EMG (VAR)`: 
    > $\frac{1}{N-1}\sum_{i=1}^{N}x_i^2$
* `Root Mean Square (RMS)`:
    > $\sqrt{\frac{1}{N-1}\sum_{i=1}^{N}|x_i|^2}$
* `Frequency Median (FMD)`:
    > $\frac{1}{2}\sum_{i=1}^{M}PSD$
* `Modified Median Frequency (MMDF)`:
    > $\frac{1}{2}\sum_{i=1}^{M}A_j$

Para a transformação dos dados no domínio da frequência, será utilizado a biblioteca `librosa`.

No final, o vetor de características estará organizado da seguinte forma:

| ID sample | VAR | RMS | FMD | MMDF | Classe |
|:---------:|:---:|:---:|:---:|:----:|:------:|
|     1     |  v1 |  v1 |  v1 |  v1  |    0   |
|     2     |  v2 |  v2 |  v2 |  v2  |    0   |
|    ...    | ... | ... | ... |  ... |   ...  |
|     N     |  vN |  vN |  vN |  vN  |    7   |

### Segmentação dos dados

In [134]:
print(X1.shape)
# definição do salto e do tamanho do segmento (segmento - salto = sobreposição)
salto = 480
segmento = 1025
n_win = int((X1.shape[-1] - segmento) / salto) + 1
ids = np.arange(n_win) * salto
x = np.array([X1[:,:,:,k:(k + segmento)] for k in ids]).transpose(1, 2, 3, 0, 4)
print(x.shape)

(8, 24, 7, 20000)
(8, 24, 7, 40, 1025)


#### Implementação das características no domínio do tempo

In [150]:
# VAR
var = np.sum(x ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1)
print(var.shape)
# RMS
rms = np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1))
print(rms.shape)

(8, 24, 7, 40)
(8, 24, 7, 40)


#### Transformação para o domínio da frequência

In [151]:
from librosa import stft

# aplicando stft no último eixo de X1 (3), com janela de 2048 e sobreposição de 512
w = np.apply_along_axis(stft, 3, X1, n_fft=2048, hop_length=512)
w = np.swapaxes(w, 3, 4)
print(w.shape)

(8, 24, 7, 40, 1025)


#### Implementação das características no domínio da frequência

In [ ]:
# FMD

# MMDF
